In [1]:
import os
os.chdir('..')

from seg_game_typehinted import SevenSegButtonGame, MiniGame
from rpi_tm1638_animations import TM1638Animated as Tm

In [2]:
from random import randint
from typing import Any, Dict, List

In [3]:
math_answer_num = randint(1, 256)
math_answer_num_str = str(math_answer_num)
math_win_length = len(math_answer_num_str)


def space_setup(tm1638: Tm) -> Dict[str, Any]:
    """
    Setup answers and starting display for space game
    """
        
    return_dict = {
        'correct_answer_conditions': ,
        'game_seg_display': ,
        'game_LED_display': 
    }
        
    return return_dict


def sapce_map_input(input_button: int,
                   game_seg_display: List[int]) -> int:
    """
    Maps the button input to corrsonding the segment display input
    """
    input_button = game_seg_display[input_button]
    print(f'selected_segment_int: {input_button}')
    
    return input_button


def space_incorrect_answer_action(tm1638: Tm) -> List[Any]:
    """
    Reset progress if incorrect for math game
    """
    print("Error!")
    return 0


space_game = MiniGame(win_length=,
                      setup_routine=,
                      map_input =,
                      incorrect_answer_action=,
                      show_button_feedback=False,
                      input_as_linear_int=True)

seg_game = SevenSegButtonGame(stb=2,
                              clk=3,
                              dio=4,
                              test_mode=True)
seg_game.register_game('space', space_game)
seg_game.setup()

SyntaxError: expression expected after dictionary key and ':' (1982400918.py, line 12)

In [ ]:
# Check math game answers (all power of 2)
space_game.correct_answer_conditions

In [ ]:
# Check the start display
space_game.game_seg_display

In [ ]:
def test_play(input):
    print(f'playing: {input}')
    space_game.play(1 << input)

In [ ]:
test_play(0)

In [ ]:
# Check all answers correct wins game
for answer in space_game.correct_answer_conditions:
    answer_pos = space_game.game_seg_display.index(answer)
    test_play(answer_pos)

In [ ]:
# Check the game stays finished
test_play(0)

In [ ]:
# from typing import Dict, List, Optional

In [4]:
from display_mocks import seg_mock

In [5]:
sevseg = seg_mock()
sevseg.print_segs(2350)

 _   _   _   _  
|_| |_| |_| |_| 
|_|.|_|.|_|.|_|.


In [6]:
sevseg.print_segs(1.1)

 _   _  
|_| |_| 
|_|.|_|.


In [7]:
sevseg.print_segs("23.5.0")

 _   _   _   _  
|_| |_| |_| |_| 
|_|.|_|.|_|.|_|.


In [8]:
sevseg.print_segs("12.3")

 _   _   _  
|_| |_| |_| 
|_|.|_|.|_|.


ToDo:
 - Add the colors to the LEDs too :)

In [ ]:
str(format(91, '08b'))

In [ ]:
i = 0
while i < 5:
    if i == 2:
        i+=1
    print(i)
    i += 1